# Setup

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
import getpass
import os
import sys

FOLDER = 'original_year_localization'

PROJECT_NAME = 'ood-prediction'
DATA_DIR = f'/nlp/scr/suzeva/data'
MODEL_DIR = '/nlp/scr/suzeva/models'

sys.path.append(f'/nlp/scr/suzeva/{PROJECT_NAME}/src')
os.environ["HF_HOME"] = '/nlp/scr/suzeva/models'
os.environ["HF_HUB"] = '/nlp/scr/suzeva/models'

CORE_LIB_DIR = f'/nlp/scr/hij/core'
RAVEL_LIB_DIR = f'/nlp/scr/hij/internal-ravel/src'
PYVENE_LIB_DIR = f'/nlp/scr/hij/pyvene'
import sys
sys.path.append(CORE_LIB_DIR)
sys.path.append(RAVEL_LIB_DIR)
sys.path.append(PYVENE_LIB_DIR)

In [40]:
import numpy as np
import random
import torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(0)

In [41]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Models

In [42]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer


model_id = "allenai/OLMo-2-0425-1B"
revision = "stage1-step10000-tokens21B"
tokenizer = AutoTokenizer.from_pretrained(
    model_id, padding_side='left', revision=revision,
    cache_dir=MODEL_DIR)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
      model_id, low_cpu_mem_usage=True, device_map='auto',
      revision=revision,
      torch_dtype=torch.bfloat16, cache_dir=MODEL_DIR)
model = model.eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.31it/s]


# Behavioral Testing

In [43]:
from generation_utils import generate_distribution_batched


zero_count_years = [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1029, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1058, 1059, 1060, 1061, 1063, 1064, 1069, 1070, 1073, 1074, 1077, 1078, 1080, 1082, 1083, 1084, 1085, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1097, 1098, 1099, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1128, 1129, 1130, 1131, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1141, 1142, 1143, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1155, 1156, 1157, 1160, 1162, 1163, 1164, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1192, 1193, 1194, 1195, 1196, 1197, 1198, 1199, 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1213, 1214, 1217, 1218, 1219, 1221, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1237, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1259, 1261, 1263, 1264, 1265, 1267, 1268, 1269, 1270, 1273, 1275, 1276, 1277, 1278, 1280, 1281, 1282, 1284, 1285, 1287, 1288, 1289, 1294, 1297, 1301, 1303, 1304, 1307, 1313, 1314, 1317, 1318, 1319, 1321, 1322, 1324, 1325, 1326, 1328, 1329, 1331, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1341, 1342, 1345, 1346, 1350, 1351, 1352, 1353, 1355, 1358, 1359, 1362, 1363, 1364, 1365, 1367, 1368, 1369, 1370, 1372, 1373, 1376, 1380, 1383, 1385, 1386, 1387, 1389, 1392, 1393, 1395, 1398, 1399, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1410, 1411, 1412, 1414, 1415, 1417, 1419, 1420, 1422, 1423, 1424, 1425, 1426, 1427, 1429, 1430, 1431, 1434, 1435, 1438, 1443, 1447, 1449, 1451, 1455, 1456, 1457, 1458, 1459, 1461, 1462, 1465, 1466, 1467, 1468, 1469, 1470, 1471, 1472, 1474, 1475, 1476, 1480, 1481, 1484, 1487, 1488, 1495, 1497, 1499, 1501, 1504, 1505, 1507, 1508, 1510, 1519, 1523, 1526, 1529, 1532, 1541, 1546, 1549, 1554, 1561, 1565, 1567, 1569, 1570, 1574, 1575, 1581, 1583, 1587, 1588, 1591, 1593, 1594, 1597, 1601, 1610, 1612, 1615, 1631, 1654, 1656, 1671, 1684, 1690, 1691, 1694, 1698, 1713, 1734, 1753, 1762, 2026, 2028, 2029, 2032, 2033, 2037, 2039, 2041, 2042, 2044, 2046, 2047, 2048, 2051, 2052, 2053, 2054, 2056, 2058, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2085, 2086, 2087, 2088, 2089, 2091, 2092, 2094, 2095, 2096, 2097, 2098, 2099, 2101, 2102, 2103, 2104, 2106, 2107, 2108, 2109, 2110, 2112, 2114, 2115, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2124, 2126, 2127, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136, 2137, 2138, 2139, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2150, 2151, 2152, 2154, 2155, 2156, 2157, 2158, 2159, 2160, 2161, 2162, 2163, 2164, 2165, 2166, 2167, 2168, 2169, 2170, 2171, 2172, 2173, 2174, 2175, 2176, 2177, 2178, 2180, 2181, 2182, 2184, 2185, 2186, 2187, 2188, 2189, 2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199, 2200, 2201, 2202, 2203, 2204, 2205, 2206, 2207, 2208, 2209, 2210, 2211, 2212, 2213, 2214, 2215, 2216, 2217, 2218, 2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2229, 2230, 2231, 2232, 2233, 2234, 2235, 2236, 2237, 2238, 2239, 2240, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 2252, 2253, 2254, 2255, 2256, 2257, 2258, 2259, 2260, 2261, 2262, 2263, 2264, 2265, 2266, 2267, 2268, 2269, 2270, 2271, 2272, 2273, 2274, 2275, 2276, 2277, 2278, 2279, 2280, 2281, 2282, 2283, 2284, 2285, 2286, 2287, 2288, 2289, 2290, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2312, 2313, 2314, 2315, 2316, 2317, 2318, 2319, 2320, 2321, 2322, 2323, 2324, 2325, 2326, 2327, 2328, 2329, 2330, 2331, 2332, 2333, 2334, 2335, 2336, 2337, 2338, 2339, 2340, 2341, 2342, 2343, 2344, 2345, 2346, 2347, 2348, 2349, 2350, 2351, 2352, 2353, 2354, 2355, 2356, 2357, 2358, 2359, 2360, 2361, 2362, 2363, 2364, 2365, 2366, 2367, 2368, 2369, 2370, 2371, 2372, 2373, 2374, 2375, 2376, 2377, 2378, 2379, 2380, 2381, 2382, 2383, 2384, 2385, 2386, 2387, 2388, 2389, 2390, 2391, 2392, 2393, 2394, 2395, 2396, 2397, 2398, 2399, 2400, 2401, 2402, 2403, 2404, 2405, 2406, 2407, 2408, 2409, 2410, 2411, 2412, 2413, 2414, 2415, 2416, 2417, 2418, 2419, 2420, 2421, 2422, 2423, 2424, 2425, 2426, 2427, 2428, 2429, 2430, 2431, 2432, 2433, 2434, 2435, 2436, 2437, 2438, 2439, 2440, 2441, 2442, 2443, 2444, 2445, 2446, 2447, 2448, 2449, 2450, 2451, 2452, 2453, 2454, 2455, 2456, 2457, 2458, 2459, 2460, 2461, 2462, 2463, 2464, 2465, 2466, 2467, 2468, 2469, 2470, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2478, 2479, 2480, 2481, 2482, 2483, 2484, 2485, 2486, 2487, 2488, 2489, 2490, 2491, 2492, 2493, 2494, 2495, 2496, 2497, 2498, 2499, 2500, 2501, 2502, 2503, 2504, 2505, 2506, 2507, 2508, 2509, 2510, 2511, 2512, 2513, 2514, 2515, 2516, 2517, 2518, 2519, 2520, 2521, 2522, 2523, 2524, 2525, 2526, 2527, 2528, 2529, 2530, 2531, 2532, 2533, 2534, 2535, 2536, 2537, 2538, 2539, 2540, 2541, 2542, 2543, 2544, 2545, 2546, 2547, 2548, 2549, 2550, 2551, 2552, 2553, 2554, 2555, 2556, 2557, 2558, 2559, 2560, 2561, 2562, 2563, 2564, 2565, 2566, 2567, 2568, 2569, 2570, 2571, 2572, 2573, 2574, 2575, 2576, 2577, 2578, 2579, 2580, 2581, 2582, 2583, 2584, 2585, 2586, 2587, 2588, 2589, 2590, 2591, 2592, 2593, 2594, 2595, 2596, 2597, 2598, 2599, 2600, 2601, 2602, 2603, 2604, 2605, 2606, 2607, 2608, 2609, 2610, 2611, 2612, 2613, 2614, 2615, 2616, 2617, 2618, 2619, 2620, 2622, 2623, 2624, 2625, 2626, 2627, 2628, 2629, 2630, 2631, 2632, 2633, 2634, 2635, 2636, 2637, 2638, 2639, 2640, 2641, 2642, 2643, 2644, 2645, 2646, 2647, 2648, 2649, 2650, 2651, 2652, 2653, 2654, 2655, 2656, 2657, 2658, 2659, 2660, 2661, 2662, 2663, 2664, 2665, 2666, 2667, 2668, 2669, 2670, 2671, 2672, 2673, 2674, 2675, 2676, 2677, 2678, 2679, 2680, 2681, 2682, 2683, 2684, 2685, 2686, 2687, 2688, 2689, 2690, 2691, 2692, 2693, 2694, 2695, 2696, 2697, 2698, 2699, 2700, 2701, 2702, 2703, 2704, 2705, 2706, 2707, 2708, 2709, 2710, 2711, 2712, 2713, 2714, 2715, 2716, 2717, 2718, 2719, 2720, 2721, 2722, 2723, 2724, 2725, 2726, 2727, 2728, 2729, 2730, 2731, 2732, 2733, 2734, 2735, 2736, 2737, 2738, 2739, 2740, 2741, 2742, 2743, 2744, 2745, 2746, 2747, 2748, 2749, 2750, 2751, 2752, 2753, 2754, 2755, 2756, 2757, 2758, 2759, 2760, 2761, 2762, 2763, 2764, 2765, 2766, 2767, 2768, 2769, 2770, 2771, 2772, 2773, 2774, 2775, 2776, 2777, 2778, 2779, 2780, 2781, 2782, 2783, 2784, 2785, 2786, 2787, 2788, 2789, 2790, 2791, 2792, 2793, 2794, 2795, 2796, 2797, 2798, 2799, 2800, 2801, 2803, 2804, 2805, 2806, 2807, 2808, 2809, 2810, 2811, 2812, 2813, 2814, 2815, 2816, 2817, 2818, 2819, 2820, 2821, 2822, 2823, 2824, 2825, 2826, 2827, 2828, 2829, 2830, 2831, 2832, 2833, 2834, 2835, 2836, 2837, 2838, 2839, 2840, 2841, 2842, 2843, 2844, 2845, 2846, 2847, 2848, 2849, 2850, 2851, 2852, 2853, 2854, 2855, 2856, 2857, 2858, 2859, 2860, 2861, 2862, 2863, 2864, 2865, 2866, 2867, 2868, 2869, 2870, 2871, 2872, 2873, 2874, 2875, 2876, 2877, 2878, 2879, 2880, 2881, 2882, 2883, 2884, 2885, 2886, 2887, 2888, 2889, 2890, 2891, 2892, 2893, 2894, 2895, 2896, 2897, 2898, 2899, 2900, 2901, 2902, 2903, 2904, 2905, 2906, 2907, 2908, 2909, 2910, 2911, 2912, 2913, 2914, 2915, 2916, 2917, 2918, 2919, 2920, 2921, 2922, 2923, 2924, 2925, 2926, 2927, 2928, 2929, 2930, 2931, 2932, 2933, 2934, 2935, 2936, 2937, 2938, 2939, 2940, 2941, 2942, 2943, 2944, 2945, 2946, 2947, 2948, 2949, 2950, 2951, 2952, 2953, 2954, 2955, 2956, 2957, 2958, 2959, 2960, 2961, 2962, 2963, 2964, 2965, 2966, 2967, 2968, 2969, 2970, 2971, 2972, 2973, 2974, 2975, 2976, 2977, 2978, 2979, 2980, 2981, 2982, 2983, 2984, 2985, 2986, 2987, 2988, 2989, 2990, 2991, 2992, 2993, 2994, 2995, 2996, 2997, 2998, 2999]
prompt_template = ['In {year}, there']
year_value = list(set(range(1000, 3000)) - set(zero_count_years))
year_value = range(1950, 2050)
print(len(year_value))
prompts = [prompt_template[0].format(year=year) for year in year_value]

100


In [44]:
outputs = generate_distribution_batched(model, tokenizer, [prompt_template[0].format(year=i) for i in range(1000, 3000)])

100%|██████████| 63/63 [00:01<00:00, 37.30it/s]


In [45]:
for dist in outputs:
  print(dist)
  break

[(' were', 0.376953125), (' are', 0.259765625), (' was', 0.111328125), (' will', 0.09521484375), (' is', 0.055908203125), (' would', 0.019287109375), (' may', 0.01416015625), ("'s", 0.006683349609375), (' have', 0.006378173828125), (' might', 0.005279541015625)]


In [46]:
len(prompts)

100

In [ ]:
# Create intervention data
import json
import os
import random

random.seed(0)
random.shuffle(prompts)

data = {
    'train': {'correct': prompts[:50], 'wrong': []},
    'val': {'correct': prompts[50:100], 'wrong': []},
    'test': {'correct': prompts[50:100], 'wrong': []},
}

# Create directory if it doesn't exist
os.makedirs(FOLDER, exist_ok=True)

# Save the data
output_file = f'{FOLDER}/year_{model_id.split("/")[-1]}-revision{revision}_in_distribution_split_0.json'
with open(output_file, 'w') as f:
    json.dump(data, f)

print(f"Data saved to: {output_file}")

Data saved to: original_year_localization/year_OLMo-2-0425-1B-revisionstage1-step10000-tokens21B_in_distribution_split_0.json


# Localizing Representations of Year

In [ ]:
import json

from data_utils import load_intervention_data, _BASE_TEMPLATE
from generation_utils import generate_batched

sample_size = 512
split_type = ''
SPLIT_ID = '1'
mode = 'das'
data_split = json.load(open(os.path.join(f'{FOLDER}/year_{model_id.split("/")[-1]}-revision{revision}_in_distribution_split_0.json')))

verified_examples = data_split['train']['correct'][:sample_size]
print(verified_examples[:2])

intervention_prompt_to_output = generate_batched(model, tokenizer, [p for s in data_split for k in ('correct', 'wrong') for p in data_split[s][k]], max_new_tokens=1)
prompt_to_vars = {p: {'input': p,
                      'label': intervention_prompt_to_output[p],
                      'split': _BASE_TEMPLATE}
                 for s in data_split for k in ('correct', 'wrong') for p in data_split[s][k]}


def get_tense(be_word):
  be_word = be_word.lower().strip()
  # We distinguish present and future here even though English does not.
  if be_word == 'will':
    return 'future'
  elif be_word == 'is' or be_word == 'are':
    return 'present'
  elif be_word.endswith('ed') or be_word == 'was' or be_word == 'were':
    return 'past'
  else:
    raise ValueError(f'Unknown tense for {be_word}')


def set_tense(be_word, tense):
  normalize_be_word = be_word.lower().strip()
  tense_table = {
      'future': {'will': 'will', 'is': 'will', 'are': 'will', 'was': 'will', 'were': 'will'},
      'present': {'will': 'is', 'is': 'is', 'are': 'are', 'was': 'is', 'were': 'are'},
      'past': {'will': 'was', 'is': 'was', 'are': 'were', 'was': 'was', 'were': 'were'},
  }
  new_be_word =  tense_table[tense][normalize_be_word]
  if be_word.startswith(' '):
    new_be_word = ' ' + new_be_word
  return new_be_word


split_to_raw_example, split_to_dataset = load_intervention_data(
    mode, verified_examples, data_split, prompt_to_vars,
    inv_label_fn=lambda x, y: set_tense(x['label'], get_tense(y['label'])),
    filter_fn=lambda x, y: random.random() < 0.005 or get_tense(x['label']) != get_tense(y['label']),
    max_example_per_split=20480,
    max_example_per_eval_split=10)

['In 1973, there', 'In 1958, there']
Total #prompts=150
Set prompt_max_length=8


100%|██████████| 5/5 [00:00<00:00, 42.01it/s]

mode=das, #base_examples=50, #source_examples=50+50+0=100
BEFORE SPLIT: #Training examples=50*50=872, #Validation examples==0, #Test examples=50*(50+0)=946


AFTER SPLIT KEPT: #Training examples=min(50*50, 20480)=872, #Validation examples==0, #Test examples=50*10+0*10=500
#Splits=1 training split plus 1,024 test splits=51


In [49]:
len(split_to_dataset['das-train'])

872

In [50]:
split_to_dataset['das-train'][6]

{'input': 'In 2030, there',
 'label': ' will',
 'source_input': 'In 1974, there',
 'source_label': ' was',
 'inv_label': ' was',
 'split': 'BASE_TEMPLATE',
 'source_split': 'BASE_TEMPLATE'}

In [ ]:
SCR_MODEL_DIR = f'/nlp/scr/suzeva/{FOLDER}'

In [52]:
import gc


gc.collect()
torch.cuda.empty_cache()

In [73]:
import collections
import gc
import re

from tqdm import tqdm, trange
from transformers import get_linear_schedule_with_warmup
from datasets import concatenate_datasets
from torch.nn import CrossEntropyLoss
from causal_interventions import compute_string_based_metrics

import pyvene as pv
from utils.intervention_utils import LowRankRotatedSpaceIntervention, get_intervention_config, train_intervention_step, remove_invalid_token_id, remove_all_forward_hooks

from utils.dataset_utils import get_multitask_dataloader
from utils.metric_utils import compute_cross_entropy_loss
from causal_interventions import eval_with_interventions_batched, compute_metrics


def train_alignment(config):
  print('Training Tasks: %s' % config['training_tasks'])
  concat_split_to_dataset = {f'joint-{split}':
      concatenate_datasets([split_to_dataset[f'{task_name}-{split}'].select(
          np.random.choice(len(split_to_dataset[f'{task_name}-{split}']),
                           size=(1024 if config['training_tasks'][task_name] == 'match_base' else len(split_to_dataset[f'{task_name}-{split}'])),
                           replace=False))
                            for task_name in config['training_tasks']
                            # repeat
                            for _ in range(1 if isinstance(config['training_tasks'][task_name], str) or split != 'train'
                                             else config['training_tasks'][task_name][1])
                            if f'{task_name}-{split}' in split_to_dataset])
      for split in ('train',)}
  inv_task = '|'.join([task_name for task_name, label in config['training_tasks'].items()
                       if label == 'match_source' or 'match_source' in label])
  inv_task = inv_task.split('|')
  print('Training tasks matching source label: %s' % inv_task)
  print('#Training examples: %d' % len(concat_split_to_dataset['joint-train']))
  max_train_example = int(config['max_train_percentage'] * len(concat_split_to_dataset['joint-train']))
  train_dataloader = get_multitask_dataloader(
      concat_split_to_dataset['joint-train'].select(range(max_train_example)),
      tokenizer=tokenizer,
      batch_size=TRAINING_BATCH_SIZE, prompt_max_length=INPUT_MAX_LEN,
      output_max_length=config['max_output_tokens'] + int(tokenizer.bos_token is not None),
      # The set of splits to load as cause tasks
      cause_tasks=[BASE_TEMPLATE, SOURCE_TEMPLATE],
      first_n=config['max_output_tokens'])


  # Create Model
  split_to_inv_locations = config['split_to_inv_locations']
  intervenable_config = get_intervention_config(
      type(model), config['intervenable_config']['intervenable_representation_type'],
      config['intervenable_config']['intervenable_layer'],
      config['intervenable_config']['intervenable_interventions_type'],
      intervention_dimension=config['intervention_dimension'])
  intervenable = pv.IntervenableModel(intervenable_config, model)
  intervenable.set_device("cuda")
  intervenable.disable_model_gradients()

  # Training
  epochs = config['training_epoch']
  gradient_accumulation_steps = 1
  total_step = 0

  warm_up_steps = 0 # 0.1 * t_total
  regularization_coefficient = config['regularization_coefficient']
  optimizer_params = []
  for k, v in intervenable.interventions.items():
      if isinstance(v[0], LowRankRotatedSpaceIntervention):
        optimizer_params += [{'params': v[0].rotate_layer.parameters()}]
      else:
        raise NotImplementedError
  optimizer = torch.optim.AdamW(
      optimizer_params, lr=config['init_lr'], weight_decay=0)
  scheduler = get_linear_schedule_with_warmup(
      optimizer, num_warmup_steps=warm_up_steps,
      num_training_steps=int(10 * len(train_dataloader))
  )

  #intervenable.model.train() # train enables drop-off but no grads
  print("base model trainable parameters: ", pv.count_parameters(intervenable.model))
  print("intervention trainable parameters: ", intervenable.count_parameters())
  train_iterator = trange(0, int(epochs), desc="Epoch")

  num_output_tokens = config['max_output_tokens']
  for epoch in train_iterator:
      epoch_iterator = tqdm(
          train_dataloader, desc=f"Epoch: {epoch}", position=0, leave=True
      )
      aggreated_stats = collections.defaultdict(list)
      for step, inputs in enumerate(epoch_iterator):
          for k, v in inputs.items():
              if v is not None and isinstance(v, torch.Tensor):
                  inputs[k] = v.to("cuda")
          b_s = inputs["input_ids"].shape[0]
          position_ids = {f'{prefix}position_ids': intervenable.model.prepare_inputs_for_generation(
                  input_ids=inputs[f"{prefix}input_ids"], attention_mask=inputs[f"{prefix}attention_mask"])['position_ids']
                  for prefix in ('', 'source_')}
          inputs.update(position_ids)
          for key in inputs:
            if key in ('input_ids', 'source_input_ids', 'attention_mask', 'source_attention_mask', 'position_ids', 'source_position_ids'):
              inputs[key] = inputs[key].to(device)

          counterfactual_outputs = train_intervention_step(
              intervenable, inputs, split_to_inv_locations, pad_token_id=tokenizer.pad_token_id)
          eval_metrics = compute_metrics(
              {'inv_outputs': [counterfactual_outputs.logits[:, -num_output_tokens-1:-1]]},
              [inputs['labels'][:, :num_output_tokens]],
              last_n_tokens=num_output_tokens,
              pad_token_id=tokenizer.pad_token_id,
          )
          loss = compute_cross_entropy_loss(
              counterfactual_outputs.logits,
              inputs["labels"][:, :num_output_tokens],
              next_n_tokens=num_output_tokens,
              pad_token_id=tokenizer.pad_token_id,
          )
          aggreated_stats['loss'].append(loss.item())
          aggreated_stats['acc'].append(eval_metrics['inv_outputs']["accuracy"])
          epoch_iterator.set_postfix({k: round(np.mean(aggreated_stats[k]), 2) for k in aggreated_stats})

          if step < 3:
            print('\nTokens to intervene:')
            intervention_locations = [split_to_inv_locations[inputs["split"][i]]['inv_position'] for i in range(len(inputs["split"]))]
            source_intervention_locations = [split_to_inv_locations[inputs["source_split"][i]]['inv_position'] for i in range(len(inputs["split"]))]
            print(inputs['input'][:3])
            print(inputs['source_input'][:3])
            print('Base:', tokenizer.batch_decode([inputs['input_ids'][i][intervention_locations[i]] for i in range(len(inputs["split"]))]))
            print('Source:', tokenizer.batch_decode([inputs['source_input_ids'][i][source_intervention_locations[i]] for i in range(len(inputs["split"]))]))
            print('Output:', tokenizer.batch_decode(torch.argmax(counterfactual_outputs.logits[:, -num_output_tokens-1:-1], dim=-1)))
            print('Label     :', tokenizer.batch_decode(remove_invalid_token_id(inputs['labels'][:, :num_output_tokens], tokenizer.pad_token_id)))
            print('Base Label:', tokenizer.batch_decode(remove_invalid_token_id(inputs['base_labels'][:, :num_output_tokens], tokenizer.pad_token_id)))

          if gradient_accumulation_steps > 1:
              loss = loss / gradient_accumulation_steps
          if total_step % gradient_accumulation_steps == 0:
              if not (gradient_accumulation_steps > 1 and total_step == 0):
                  loss.backward()
                  optimizer.step()
                  scheduler.step()
                  intervenable.set_zero_grad()
          total_step += 1
  return intervenable, intervenable_config


def run_exp(config):
  inv_tasks = '+'.join([''.join(re.findall(r'[A-Za-z]+', t)) + ('' if isinstance(l, str) else str(l[1])) for t, l in config['training_tasks'].items() if l == 'match_source' or 'match_source' in l])
  control_tasks = '+'.join([''.join(re.findall(r'[A-Za-z]+', t)) for t, l in config['training_tasks'].items() if l == 'match_base' or 'match_base' in l])
  task_compressed = ((inv_tasks + '_ex_' + control_tasks) if control_tasks else inv_tasks).replace('AZaz', '')
  das_type = 'multi_das' if len(config['training_tasks']) > 1 else 'das_baseline'
  if config['intervenable_config']['intervenable_interventions_type'] == LowRankRotatedSpaceIntervention:
    das_type = das_type.replace('das', 'daslora')
  split_to_inv_locations = config['split_to_inv_locations']
  input_len = list(split_to_inv_locations.values())[0]['max_input_length']
  inv_pos = min([x['inv_position'][0] for x in split_to_inv_locations.values()])
  inv_loc_name = 'len%d_pos%s' % (input_len, 'e' if inv_pos != input_len - 1 else 'f')
  training_data_percentage = int(config['max_train_percentage'] * 100)
  suffix = f"_example{len(verified_examples)}_{config['intervenable_config']['intervenable_representation_type']}"
  layer = '%s_%s' % (min(config['intervenable_config']['intervenable_layer']), max(config['intervenable_config']['intervenable_layer'])) if isinstance(config['intervenable_config']['intervenable_layer'], list) else config['intervenable_config']['intervenable_layer']
  model_name = model.name_or_path.split('/')[-1] + f'_{revision}'
  run_name = f"{model_name}-layer{layer}-dim{config['intervention_dimension']}-{das_type}_{config['max_output_tokens']}tok_{task_compressed}_id-{SPLIT_ID}_{inv_loc_name}_ep{config['training_epoch']}{suffix}"
  config['run_name_prefix'] = run_name#.rsplit('_ep', 1)[0]
  print(run_name)
  log_file_path = os.path.join(SCR_MODEL_DIR, 'logs', f'{run_name}.log')
  if True:
      print(run_name)
      intervenable, intervenable_config = train_alignment(config)
      # Save model
      torch.save({k: v[0].rotate_layer.weight for k, v in intervenable.interventions.items()},
                 os.path.join(SCR_MODEL_DIR, f'{run_name}.pt'))
      print('Model saved to %s' % os.path.join(SCR_MODEL_DIR, f'{run_name}.pt'))
      gc.collect()
      torch.cuda.empty_cache()
      # eval
      with torch.no_grad():
        split_to_eval_metrics = eval_with_interventions_batched(
            intervenable, eval_split_to_dataset,
            split_to_inv_locations,
            tokenizer,
            compute_metrics_fn=compute_metrics,
            max_new_tokens=config['max_output_tokens'],
            eval_batch_size=EVAL_BATCH_SIZE,
            inference_mode='generate',
            debug_print=True,
          )
      print('Mean IIA: %.4f' % np.mean(
          [v['metrics']['labels']['inv_outputs']['accuracy'] for k, v in split_to_eval_metrics.items()]))
      print('Mean correct IIA: %.4f' % np.mean(
          [v['metrics']['labels']['inv_outputs']['accuracy'] for k, v in split_to_eval_metrics.items() if '-correct' in k]))
      print('Mean wrong IIA: %.4f' % np.mean(
          [v['metrics']['labels']['inv_outputs']['accuracy'] for k, v in split_to_eval_metrics.items() if '-wrong' in k]))
  # Save model.
  #torch.save({k: v[0].rotate_layer.weight for k, v in intervenable.interventions.items()},
  #           os.path.join(SCR_MODEL_DIR, f'{config["run_name_prefix"]}.pt'))
  #print('Model saved to %s' % os.path.join(SCR_MODEL_DIR, f'{config["run_name_prefix"]}.pt'))
  # logging
  json.dump(split_to_eval_metrics, open(os.path.join(SCR_MODEL_DIR, f'{run_name}_evalall.json'), 'w'))
  print('Saved to %s' % os.path.join(SCR_MODEL_DIR, f'{run_name}.json'))
  remove_all_forward_hooks(intervenable)
  return intervenable



assert mode == 'das'

INPUT_MAX_LEN = 8
TRAINING_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 16

from data_utils import _BASE_TEMPLATE, _SOURCE_TEMPLATE
BASE_TEMPLATE = _BASE_TEMPLATE
SOURCE_TEMPLATE = _SOURCE_TEMPLATE

SPLIT_TO_INV_LOCATIONS = {
    split: {'max_input_length': INPUT_MAX_LEN,
             'inv_position': [INPUT_MAX_LEN - 3]}
    for split in list(split_to_dataset) + [BASE_TEMPLATE, SOURCE_TEMPLATE]
}

training_tasks_list = [
  {'das': 'match_source'}
]

eval_split_to_dataset = {k: v for k, v in split_to_dataset.items()
                         if k.endswith('-test')
                         }

model = model.eval()



for inv_layer in [[i] for i in range(10)]:
  for lr in [1e-4]:
    for inv_dim in [16, 32]:
      # train
      for training_tasks in training_tasks_list:
        config = {
            'regularization_coefficient': 0,
            'intervention_dimension': inv_dim,
            'max_output_tokens': 1,
            'intervenable_config': {
              'intervenable_layer': inv_layer,
              'intervenable_representation_type': 'block_output',
              'intervenable_unit': 'pos',
              'max_number_of_units': 1,
              'intervenable_interventions_type': LowRankRotatedSpaceIntervention,
            },
            'training_tasks': training_tasks,
            'training_epoch': 1,
            'split_to_inv_locations': SPLIT_TO_INV_LOCATIONS,
            'split_to_labels': None,
            'max_train_percentage': 1.0 if len(training_tasks) <= 3 else 1.0,
            'init_lr': lr,
        }
        intervenable = run_exp(config)

OLMo-2-0425-1B_stage1-step10000-tokens21B-layer0_0-dim16-daslora_baseline_1tok_das_id-1_len8_pose_ep1_example50_block_output
OLMo-2-0425-1B_stage1-step10000-tokens21B-layer0_0-dim16-daslora_baseline_1tok_das_id-1_len8_pose_ep1_example50_block_output
Training Tasks: {'das': 'match_source'}
Training tasks matching source label: ['das']
#Training examples: 872


Map: 100%|██████████| 872/872 [00:00<00:00, 15300.38 examples/s]


base model trainable parameters:  0
intervention trainable parameters:  32768


Epoch: 0:   4%|▎         | 2/54 [00:00<00:03, 14.15it/s, loss=4.13, acc=0]


Tokens to intervene:
['In 2036, there', 'In 1958, there', 'In 2049, there']
['In 1958, there', 'In 2028, there', 'In 1998, there']
Base: ['6', '8', '9', '0', '6', '3', '6', '0', '9', '3', '6', '8', '4', '3', '7', '9']
Source: ['8', '8', '8', '6', '8', '9', '8', '5', '6', '4', '1', '0', '0', '7', '9', '4']
Output: [' will', ' was', ' will', ' were', ' was', ' was', ' will', ' will', ' was', ' was', ' was', ' was', ' was', ' will', ' will', ' will']
Label     : [' was', ' will', ' was', ' will', ' will', ' will', ' was', ' was', ' will', ' will', ' will', ' will', ' will', ' was', ' was', ' was']
Base Label: [' will', ' was', ' will', ' were', ' was', ' was', ' will', ' will', ' was', ' was', ' was', ' was', ' was', ' will', ' will', ' will']

Tokens to intervene:
['In 1978, there', 'In 2043, there', 'In 2017, there']
['In 2037, there', 'In 1954, there', 'In 2049, there']
Base: ['8', '3', '7', '8', '7', '1', '2', '4', '4', '0', '7', '4', '5', '0', '1', '1']
Source: ['7', '4', '9', '6', 

...


 'source-In 1983, there-correct-test': {'base_labels': {'base_outputs': {'accuracy': 1.0, 'token_accuracy': 1.0, 'class_0_accuracy': 1.0}, 'inv_outputs': {'accuracy': 0.0, 'token_accuracy': 0.0, 'class_0_accuracy': 0.0}, 'accuracy': 0.0}, 'labels': {'base_outputs': {'accuracy': 0.0, 'token_accuracy': 0.0, 'class_0_accuracy': 0.0}, 'inv_outputs': {'accuracy': 0.7, 'token_accuracy': 0.7, 'class_0_accuracy': 0.7}, 'accuracy': 0.7}}

 'source-In 1955, there-correct-test': {'base_labels': {'base_outputs': {'accuracy': 1.0, 'token_accuracy': 1.0, 'class_0_accuracy': 0.9}, 'inv_outputs': {'accuracy': 0.1, 'token_accuracy': 0.1, 'class_0_accuracy': 0.0}, 'accuracy': 0.1}, 'labels': {'base_outputs': {'accuracy': 0.1, 'token_accuracy': 0.1, 'class_0_accuracy': 0.0}, 'inv_outputs': {'accuracy': 0.7, 'token_accuracy': 0.7, 'class_0_accuracy': 0.6}, 'accuracy': 0.7}}


Test:  94%|█████████▍| 30/32 [00:22<00:01,  1.44it/s]


 'source-In 2003, there-correct-test': {'base_labels': {'base_outputs': {'accuracy': 1.0, 'token_accuracy': 1.0, 'class_0_accuracy': 1.0}, 'inv_outputs': {'accuracy': 0.0, 'token_accuracy': 0.0, 'class_0_accuracy': 0.0}, 'accuracy': 0.0}, 'labels': {'base_outputs': {'accuracy': 0.0, 'token_accuracy': 0.0, 'class_0_accuracy': 0.0}, 'inv_outputs': {'accuracy': 0.7, 'token_accuracy': 0.7, 'class_0_accuracy': 0.7}, 'accuracy': 0.7}}

 'source-In 2047, there-correct-test': {'base_labels': {'base_outputs': {'accuracy': 1.0, 'token_accuracy': 1.0, 'class_0_accuracy': 0.5}, 'inv_outputs': {'accuracy': 0.0, 'token_accuracy': 0.0, 'class_0_accuracy': 0.0}, 'accuracy': 0.0}, 'labels': {'base_outputs': {'accuracy': 0.0, 'token_accuracy': 0.0, 'class_0_accuracy': 0.0}, 'inv_outputs': {'accuracy': 1.0, 'token_accuracy': 1.0, 'class_0_accuracy': 1.0}, 'accuracy': 1.0}}


Test: 100%|██████████| 32/32 [00:23<00:00,  1.34it/s]


 'source-In 1999, there-correct-test': {'base_labels': {'base_outputs': {'accuracy': 1.0, 'token_accuracy': 1.0, 'class_0_accuracy': 1.0}, 'inv_outputs': {'accuracy': 0.0, 'token_accuracy': 0.0, 'class_0_accuracy': 0.0}, 'accuracy': 0.0}, 'labels': {'base_outputs': {'accuracy': 0.0, 'token_accuracy': 0.0, 'class_0_accuracy': 0.0}, 'inv_outputs': {'accuracy': 0.8, 'token_accuracy': 0.8, 'class_0_accuracy': 0.8}, 'accuracy': 0.8}}
Mean IIA: 0.8000
Mean correct IIA: 0.8000
Mean wrong IIA: nan
Saved to /nlp/scr/suzeva/original_year_localization/OLMo-2-0425-1B_stage1-step10000-tokens21B-layer9_9-dim32-daslora_baseline_1tok_das_id-1_len8_pose_ep1_example50_block_output.json


In [74]:
# OLMo-2-0425-1B_stage1-step10000-tokens21B-layer9_9-dim8-daslora_baseline_1tok_das_id-1_len8_pose_ep1_example500_block_output


# Accuracy
inv_dims = [1, 2, 4, 8, 16, 32]

print(f'inv dim: {inv_dims}')
for l in range(10):
  iia = []
  for d in inv_dims:  # OLMo-2-0425-1B_stage1-step10000-tokens21B-layer0_0-dim1-daslora_baseline_1tok_das_id-1_len8_pose_ep1_example50_block_output_evalall
    split_to_eval_metrics = json.load(open(f'/nlp/scr/suzeva/{FOLDER}/OLMo-2-0425-1B_stage1-step10000-tokens21B-layer{l}_{l}-dim{d}-daslora_baseline_1tok_das_id-1_len8_pose_ep1_example50_block_output_evalall.json'))
    iia.append(np.mean([v['metrics']['labels']['inv_outputs']['accuracy']
                        for k, v in split_to_eval_metrics.items() if '-correct' in k]).tolist())
  print(f'layer={l}\t' + '\t'.join(map(lambda x: f'{x:.2f}', iia)))

inv dim: [1, 2, 4, 8, 16, 32]
layer=0	0.01	0.10	0.13	0.16	0.19	0.26
layer=1	0.06	0.05	0.17	0.19	0.32	0.42
layer=2	0.23	0.27	0.49	0.52	0.53	0.53
layer=3	0.36	0.51	0.53	0.53	0.56	0.67
layer=4	0.45	0.49	0.54	0.62	0.90	0.96
layer=5	0.31	0.51	0.54	0.69	0.91	0.97
layer=6	0.48	0.51	0.55	0.73	0.90	0.97
layer=7	0.38	0.50	0.54	0.75	0.90	0.97
layer=8	0.33	0.51	0.55	0.72	0.87	0.93
layer=9	0.34	0.51	0.52	0.70	0.79	0.80


# PCA

In [75]:
import collections
import gc
import json
import numpy as np
import re

import h5py
from utils.intervention_utils import remove_all_forward_hooks


def get_last_token_activations(activation_store, repr_type):
  def hook(model, input, output):
    print(f"\n=== Hook called for {repr_type} ===")
    
    if repr_type == 'attention_weight':
      # attention otuput: attention_output, present, (attention_weights)
      print(f"Attention weight output type: {type(output)}, len: {len(output) if hasattr(output, '__len__') else 'N/A'}")
      if len(output) > 2:
        print(f"output[2] shape: {output[2].shape}")
      repr = output[2].detach().cpu()[:, :, -1, :]
    elif repr_type == 'attention_output':
      # attention otuput: attention_output, present, (attention_weights)
      print(f"Attention output type: {type(output)}, len: {len(output) if hasattr(output, '__len__') else 'N/A'}")
      print(f"output[0] shape: {output[0].shape}")
      repr = output[0].detach().cpu()[:, -1, :]
    elif repr_type == 'block_output':
      print(f"Block output type: {type(output)}")
      if isinstance(output, (tuple, list)):
        print(f"Block output len: {len(output)}")
        print(f"output[0] shape: {output[0].shape}")
        print(f"output[0] dtype: {output[0].dtype}")
      else:
        print(f"Block output shape: {output.shape}")
        print(f"Block output dtype: {output.dtype}")
      
      if isinstance(output, (tuple, list)):
        repr = output[0].detach().cpu()[:, -1, :]
      else:
        repr = output.detach().cpu()[:, -1, :]
        
    elif repr_type == 'mlp_output':
      print(f"MLP output shape: {output.shape}")
      print(f"MLP output dtype: {output.dtype}")
      repr = output.detach().cpu()[:, -1, :]
    elif repr_type == 'mlp_input':
      print(f"MLP input type: {type(input)}, len: {len(input) if hasattr(input, '__len__') else 'N/A'}")
      print(f"input[0] shape: {input[0].shape}")
      repr = input[0].detach().cpu()[:, -1, :]
    elif repr_type == 'attention_c_attn_output':
      print(f"Attention c_attn output shape: {output.shape}")
      repr = output.detach().cpu()
    
    print(f"Final repr shape: {repr.shape}")
    print(f"=== End hook for {repr_type} ===\n")
    activation_store.append(repr)
  return hook


def extract_activations_batched(text_inputs, pos, layer, input_max_length, batch_size=128, subspace=None):
  remove_all_forward_hooks(model)
  # Extract residual representations at the last token.
  repr_type = 'block_output'
  
  print(f"\n=== Starting extract_activations_batched ===")
  print(f"Number of text inputs: {len(text_inputs)}")
  print(f"pos: {pos}, layer: {layer}, input_max_length: {input_max_length}, batch_size: {batch_size}")
  print(f"First few text inputs: {text_inputs[:3]}")
  
  initial_shape = tokenizer(text_inputs, padding='longest', return_tensors='pt').input_ids.shape
  print(f"Initial tokenized shape (padding='longest'): {initial_shape}")
  
  encoded_inputs = tokenizer(
          text_inputs, return_tensors='pt', padding='max_length',
          max_length=input_max_length).to(device)
  
  print(f"Encoded inputs shape before truncation: {encoded_inputs.input_ids.shape}")
  
  # Remove the suffix so the target token is the last token.
  for k in encoded_inputs:
    encoded_inputs[k] = encoded_inputs[k][:, :pos + 1]
  
  print(f"Encoded inputs shape after truncation to pos+1={pos+1}: {encoded_inputs.input_ids.shape}")
  print(f"Sample input_ids[0]: {encoded_inputs.input_ids[0]}")
  print(f"Sample attention_mask[0]: {encoded_inputs.attention_mask[0]}")

  all_features = []
  remove_all_forward_hooks(model)
  # Extract residual representations at the last token.
  layer_features = []
  if repr_type == 'attention_weight':
    hook = model.model.layers[layer].self_attn.register_forward_hook(
        get_last_token_activations(layer_features, repr_type))
  elif repr_type == 'block_output':
    print(f"Registering hook on model.model.layers[{layer}]")
    hook = model.model.layers[layer].register_forward_hook(
        get_last_token_activations(layer_features, repr_type))
  
  with torch.no_grad():
    for b_i in range(0, len(encoded_inputs.input_ids), batch_size):
      print(f"\n--- Processing batch {b_i//batch_size + 1} (indices {b_i}:{b_i+batch_size}) ---")
      current_batch_size = min(batch_size, len(encoded_inputs.input_ids) - b_i)
      print(f"Current batch size: {current_batch_size}")
      print(f"Input batch shape: {encoded_inputs.input_ids[b_i:b_i+batch_size].shape}")
      print(f"Attention mask batch shape: {encoded_inputs.attention_mask[b_i:b_i+batch_size].shape}")
      
      _ = model.generate(input_ids=encoded_inputs.input_ids[b_i:b_i+batch_size],
                         attention_mask=encoded_inputs.attention_mask[b_i:b_i+batch_size],
                         max_new_tokens=1,
                         pad_token_id=tokenizer.pad_token_id)
      
      print(f"layer_features length after generation: {len(layer_features)}")
      if layer_features:
        concatenated = torch.cat(layer_features, dim=0)
        print(f"Concatenated layer_features shape: {concatenated.shape}")
        all_features.append(concatenated.tolist())
      layer_features.clear()
      
  hook.remove()
  del hook
  remove_all_forward_hooks(model)
  gc.collect()
  torch.cuda.empty_cache()
  
  print(f"\nall_features length: {len(all_features)}")
  if all_features:
    print(f"all_features[0] shape: {np.array(all_features[0]).shape}")
  
  all_features = np.concatenate(all_features)
  print(f"Final all_features shape: {all_features.shape}")
  
  if subspace is not None:
    subspace_key = list(subspace.keys())[0]
    print(f"Subspace key: {subspace_key}")
    print(f"Subspace shape: {subspace[subspace_key].shape}")
    subspace_features = all_features @ subspace[subspace_key].T.detach().cpu().numpy()
  else:
    subspace_features = all_features
  print('Project to subspace:', subspace_features.shape)
  del encoded_inputs
  gc.collect()
  torch.cuda.empty_cache()
  return subspace_features

# highest IIA
layer, dim = 5, 32

das_subspace = torch.load(f'/nlp/scr/suzeva/{FOLDER}/OLMo-2-0425-1B_stage1-step10000-tokens21B-layer{l}_{l}-dim{d}-daslora_baseline_1tok_das_id-1_len8_pose_ep1_example50_block_output.pt')
input_max_length = 8
batch_size = 8
layer = 4
pos = input_max_length - 3

all_prompts = [prompt_template[0].format(year=y) for y in range(1000, 3000)]
subspace_features = extract_activations_batched(
          all_prompts,
          pos=pos,
          layer=layer,
          input_max_length=input_max_length,
          batch_size=batch_size,
          subspace=das_subspace)


=== Starting extract_activations_batched ===
Number of text inputs: 2000
pos: 5, layer: 4, input_max_length: 8, batch_size: 8
First few text inputs: ['In 1000, there', 'In 1001, there', 'In 1002, there']
Initial tokenized shape (padding='longest'): torch.Size([2000, 6])
Encoded inputs shape before truncation: torch.Size([2000, 8])
Encoded inputs shape after truncation to pos+1=6: torch.Size([2000, 6])
Sample input_ids[0]: tensor([100257, 100257,    644,    220,   1041,     15], device='cuda:0')
Sample attention_mask[0]: tensor([0, 0, 1, 1, 1, 1], device='cuda:0')
Registering hook on model.model.layers[4]

--- Processing batch 1 (indices 0:8) ---
Current batch size: 8
Input batch shape: torch.Size([8, 6])
Attention mask batch shape: torch.Size([8, 6])

=== Hook called for block_output ===
Block output type: <class 'torch.Tensor'>
Block output shape: torch.Size([8, 6, 2048])
Block output dtype: torch.bfloat16
Final repr shape: torch.Size([8, 2048])
=== End hook for block_output ===

lay

In [ ]:


# PCA & cluster

import numpy as np
from sklearn.decomposition import PCA


Xs = subspace_features
Ys = [{'year': i + 1000,
       'frequency': all_prompts[i] in data_split['train']['correct'],
       'pred_past': sum([s[1] for s in outputs[i] if s[0] in (' was', ' were')]),
       'pred_future': sum([s[1] for s in outputs[i] if s[0] in (' is', ' are', ' will')]),
       'pred_diff': sum([s[1] for s in outputs[i] if s[0] in (' was', ' were')]) - sum([s[1] for s in outputs[i] if s[0] in (' is', ' are', ' will')])}
      for i in range(len(Xs))]
colors =  [k for k in Ys]
X = (Xs - np.mean(Xs, axis=0, keepdims=True)) / Xs.std(axis=0)
print(X.mean(), X.std())
pca = PCA(n_components=4)
pca.fit(X)

X_reduced = pca.transform(X)
print(X_reduced.shape, len(Ys))
print(pca.explained_variance_ratio_, np.sum(pca.explained_variance_ratio_))

3.8546943414985435e-16 1.0000000000000002
(2000, 4) 2000
[0.29950969 0.10718914 0.08127037 0.06603004] 0.5539992376554456


In [87]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import pandas as pd


def plot_pca_2d(vectors, color_labels, hue, style, rerun_pca=False):
  global pca
  figsize = (8, 5)
  is_large_plot = figsize[0] * figsize[1] > 30
  plt.rcParams['figure.dpi'] = 300 if is_large_plot else 100
  plt.rcParams['savefig.dpi'] = 300 if is_large_plot else 100
  plt.rc('font', **{'size': 6})
  plt.figure(figsize=figsize)

  if rerun_pca:
    X = vectors
    X = (X - np.mean(X, axis=0, keepdims=True)) / X.var(axis=0)**0.5
    pca = PCA(n_components=2)
    pca.fit(X)

  X_2d = pca.transform(vectors)
  fig, ax = plt.subplots()
  pc_index = [0, 1]
  data = pd.DataFrame([dict(list({f'pc_{d}': X_2d[i][d] for d in range(len(X_2d[i]))}.items()) +
                                          list(color_labels[i].items()))
                                     for i in range(len(X_2d))])
  sns.scatterplot(data=data,
                  x=f"pc_{pc_index[0]}", y=f"pc_{pc_index[1]}", hue=hue, style=style, ax=ax,
                  #palette=sns.color_palette("mako", as_cmap=True)
                  )
  # Annotate data point
  for i in range(0, data.shape[0]):
    # Annotate outliers.
    # These values are hard-coded based on the actual plot.
    if (data[f"pc_{pc_index[0]}"][i] > 4.0 or data[f"pc_{pc_index[1]}"][i] > 3.5 or
        data[f"pc_{pc_index[0]}"][i] < -2.7 or data[f"pc_{pc_index[1]}"][i] < -2.0):
      ax.text(data[f"pc_{pc_index[0]}"][i] - 0.01, data[f"pc_{pc_index[1]}"][i]- 0.01,
              data['year'][i], fontsize=6, color='white', fontweight='bold')
      ax.text(data[f"pc_{pc_index[0]}"][i], data[f"pc_{pc_index[1]}"][i],
              data['year'][i], fontsize=6)

  plt.title('Hidden Representations')
  plt.xlabel(f'Principal Component {pc_index[0] + 1}')
  plt.ylabel(f'Principal Component {pc_index[1] + 1}')
  
  # save the plot
  safe_model_id = model_id.replace('/', '_')
  plt.savefig(f'{safe_model_id}_{revision}_pca_{hue}_{style}_layer{l}_dim{d}.png')
  plt.close()

  return X_2d

In [88]:
plot_pca_2d(X, colors, hue='pred_diff', style='frequency')

array([[-0.08051292, -4.41756373, -2.00730782, -0.62249469],
       [ 2.44598896, -5.26863018, -2.04760125, -0.21617198],
       [ 2.06247985, -4.51187507, -2.53785955,  2.03584621],
       ...,
       [-3.07513631,  1.3689811 , -0.67038972, -1.38367814],
       [-3.33637004,  1.32143246, -1.37401658, -1.1960616 ],
       [-3.90143788,  1.64189494, -0.32130079, -1.84068863]],
      shape=(2000, 4))

<Figure size 2400x1500 with 0 Axes>

In [89]:
plot_pca_2d(X, colors, hue='year', style='frequency')

array([[-0.08051292, -4.41756373, -2.00730782, -0.62249469],
       [ 2.44598896, -5.26863018, -2.04760125, -0.21617198],
       [ 2.06247985, -4.51187507, -2.53785955,  2.03584621],
       ...,
       [-3.07513631,  1.3689811 , -0.67038972, -1.38367814],
       [-3.33637004,  1.32143246, -1.37401658, -1.1960616 ],
       [-3.90143788,  1.64189494, -0.32130079, -1.84068863]],
      shape=(2000, 4))

<Figure size 2400x1500 with 0 Axes>